In [24]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob

from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [25]:
directory = '~/PycharmProjects/tfm_hugopobil'

btc_usd_grouped = pd.read_csv(f'{directory}/data/sampled_data/btc_usd_grouped_v2.csv')
tweets_grouped = pd.read_csv(f'{directory}/data/sampled_data/tweets_grouped_v2.csv')

In [26]:
df_clean = pd.read_csv(f'{directory}/data/sampled_data/tweets_clean_v2.csv')
df_clean.head()

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,sample_date
0,21519,Rahul Chahal,NaN,#Bitcoin #BTC,2009-03-26 10:41:20,100.0,388.0,3401.0,False,2021-02-05 10:53:49,Bitcoin and ETH both have bullish setups for a...,"['Bitcoin', 'ETH', 'BTC']",Twitter for iPad,False,2021-02-05
1,21507,Iconic Funds,"Frankfurt, Germany",Professional Crypto Asset Management\nhttps://...,2017-08-03 10:44:25,16813.0,818.0,1201.0,False,2021-02-05 11:00:24,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,"['CryptoOutlook', 'cryptocurrency', 'bitcoin',...",Twitter Web App,False,2021-02-05
2,21491,CryptoSquawk,Australia,24x7 Crypto market real-time audio squawk for ...,2017-10-05 10:13:09,1282.0,25.0,72.0,False,2021-02-05 11:14:46,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,"['Bitcoin', 'BTC', 'crypto']",CryptoSquawkBot,False,2021-02-05
3,21483,Drew MacMartin,NaN,"Co-Founder Wealth Playbook, Macro Economics & ...",2016-03-05 15:34:35,73.0,122.0,274.0,False,2021-02-05 11:20:02,"If hyperinflation does hit again, think of the...","['hyperinflation', 'inflation', 'flood', 'GOLD...",Twitter for iPhone,False,2021-02-05
4,21443,DeriBot.info,"Chicago, IL",https://t.co/HsKyGWNVqh - The Fastest Cloud Se...,2013-07-27 07:40:54,1973.0,127.0,631.0,False,2021-02-05 11:45:50,DeriBot Daily Trading Report 5.02.2021 11:42 U...,['Bitcoin'],Twitter Web App,False,2021-02-05


In [27]:
df_clean.dropna(subset=['hashtags'], inplace=True)
df = df_clean[['text']]
df.columns = ['tweets']
df.head()

,tweets
0,Bitcoin and ETH both have bullish setups for a...
1,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...
2,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...
3,"If hyperinflation does hit again, think of the..."
4,DeriBot Daily Trading Report 5.02.2021 11:42 U...


In [28]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words(['english'])

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [29]:
lem = WordNetLemmatizer()

In [30]:
def cleaning(data):
    tweet_without_url = re.sub(r'http\S+',' ', data)
    tweet_without_hashtag = re.sub(r'#\w+', ' ', tweet_without_url)
    tweet_without_mentions = re.sub(r'@\w+',' ', tweet_without_hashtag)
    precleaned_tweet = re.sub('[^A-Za-z]+', ' ', tweet_without_mentions)

    # Tokenization
    tweet_tokens = TweetTokenizer().tokenize(precleaned_tweet)
    tokens_without_punc = [w for w in tweet_tokens if w.isalpha()]
    tokens_without_sw = [t for t in tokens_without_punc if t not in stop_words]

    text_cleaned = [lem.lemmatize(t) for t in tokens_without_sw]

    return " ".join(text_cleaned)

In [31]:
df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
df['date'] = df_clean['date']
df['date_clean'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.drop(columns='date',inplace=True)
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/2555527066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/2555527066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df_clean['date']
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/2555527066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,tweets,cleaned_tweets,date_clean
0,Bitcoin and ETH both have bullish setups for a...,Bitcoin ETH bullish setup move higher BTC woul...,2021-02-05
1,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,Bloomberg LP CryptoOutlook cryptocurrency bitc...,2021-02-05
2,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,BTC SELLING PRESSURE ALERT Price trading aroun...,2021-02-05
3,"If hyperinflation does hit again, think of the...",If hyperinflation hit think inflation like flo...,2021-02-05
4,DeriBot Daily Trading Report 5.02.2021 11:42 U...,DeriBot Daily Trading Report UTC Bitcoin Tradi...,2021-02-05


In [32]:
def get_Subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

def get_Polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

In [33]:
btc_usd_grouped = btc_usd_grouped.set_index('Date')

In [34]:
btc_usd_grouped

,Adj Close
Date,
2021-02-06,39266.011719
2021-02-07,38903.441406
2021-02-08,46196.464844
2021-02-09,46481.105469
2021-02-10,44918.183594
...,...
2022-01-24,36654.328125
2022-01-25,36954.003906
2022-01-26,36852.121094


In [35]:
def btc_price_cate(score):
    if score < 1:
        return 'negative'
    elif score == 1:
        return 'neutral'
    else:
        return 'positive'


def observe_period(period):
    res = btc_usd_grouped['Adj Close'].shift(period)/btc_usd_grouped['Adj Close']
    res = res.apply(btc_price_cate)
    return res

In [36]:
time_sentiment = observe_period(30)

In [37]:
# CHECK
df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/2522087232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)


In [38]:
df['subjectivity'] = df['cleaned_tweets'].apply(get_Subjectivity)
df['polarity'] = df['cleaned_tweets'].apply(get_Polarity)
print(df.shape)
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/816313075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subjectivity'] = df['cleaned_tweets'].apply(get_Subjectivity)


(22120, 6)


/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/816313075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity'] = df['cleaned_tweets'].apply(get_Polarity)


,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity
0,Bitcoin and ETH both have bullish setups for a...,Bitcoin ETH bullish setup move higher BTC woul...,2021-02-05,NaN,0.416667,0.350000
1,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,Bloomberg LP CryptoOutlook cryptocurrency bitc...,2021-02-05,NaN,0.000000,0.000000
2,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,BTC SELLING PRESSURE ALERT Price trading aroun...,2021-02-05,NaN,0.000000,0.000000
3,"If hyperinflation does hit again, think of the...",If hyperinflation hit think inflation like flo...,2021-02-05,NaN,0.541667,-0.291667
4,DeriBot Daily Trading Report 5.02.2021 11:42 U...,DeriBot Daily Trading Report UTC Bitcoin Tradi...,2021-02-05,NaN,0.000000,0.000000


In [39]:
def getSentiment(score, threshold=0.5):
    if score < 0:
        return 'negative'
    elif 0.001 < score < threshold:
        return 'neutral'
    else:
        return 'positive'

df['sentiment'] = df['polarity'].apply(getSentiment)
df['target'] = df['sentiment'] == df['crypto_sentiment']
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/2582751764.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['polarity'].apply(getSentiment)
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_46374/2582751764.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df['sentiment'] == df['crypto_sentiment']


,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity,sentiment,target
0,Bitcoin and ETH both have bullish setups for a...,Bitcoin ETH bullish setup move higher BTC woul...,2021-02-05,NaN,0.416667,0.350000,neutral,False
1,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,Bloomberg LP CryptoOutlook cryptocurrency bitc...,2021-02-05,NaN,0.000000,0.000000,positive,False
2,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,BTC SELLING PRESSURE ALERT Price trading aroun...,2021-02-05,NaN,0.000000,0.000000,positive,False
3,"If hyperinflation does hit again, think of the...",If hyperinflation hit think inflation like flo...,2021-02-05,NaN,0.541667,-0.291667,negative,False
4,DeriBot Daily Trading Report 5.02.2021 11:42 U...,DeriBot Daily Trading Report UTC Bitcoin Tradi...,2021-02-05,NaN,0.000000,0.000000,positive,False


In [40]:
df.target.value_counts(normalize=True)

False    0.686844
True     0.313156
Name: target, dtype: float64

In [41]:
df.crypto_sentiment.value_counts()

positive    11232
negative     9333
Name: crypto_sentiment, dtype: int64

In [42]:
df.sentiment.value_counts(normalize=True)

positive    0.543987
neutral     0.346564
negative    0.109448
Name: sentiment, dtype: float64

In [43]:
df = df.set_index('date_clean')

In [44]:
df

,tweets,cleaned_tweets,crypto_sentiment,subjectivity,polarity,sentiment,target
date_clean,,,,,,,
2021-02-05,Bitcoin and ETH both have bullish setups for a...,Bitcoin ETH bullish setup move higher BTC woul...,NaN,0.416667,0.350000,neutral,False
2021-02-05,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,Bloomberg LP CryptoOutlook cryptocurrency bitc...,NaN,0.000000,0.000000,positive,False
2021-02-05,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,BTC SELLING PRESSURE ALERT Price trading aroun...,NaN,0.000000,0.000000,positive,False
2021-02-05,"If hyperinflation does hit again, think of the...",If hyperinflation hit think inflation like flo...,NaN,0.541667,-0.291667,negative,False
2021-02-05,DeriBot Daily Trading Report 5.02.2021 11:42 U...,DeriBot Daily Trading Report UTC Bitcoin Tradi...,NaN,0.000000,0.000000,positive,False
...,...,...,...,...,...,...,...
2022-03-02,AN URGENT MESSAGE TO CRYPTO INVESTORS. Buy Bit...,AN URGENT MESSAGE TO CRYPTO INVESTORS Buy Bitc...,NaN,0.000000,0.000000,positive,False
2022-03-02,Who’s gonna tell him? 👀 btc \n\n/ObAOkIDOvX,Who gonna tell btc ObAOkIDOvX,NaN,0.000000,0.000000,positive,False
2022-03-02,I’ve been paid 2 times in btc ever in my life ...,I paid time btc ever life time client regrette...,NaN,0.700000,0.800000,positive,False


In [45]:
df.to_csv(f'{directory}/data/sampled_data/tweets_nlp_modelling_v2.csv')